<a href="https://colab.research.google.com/github/srishtiprasanna/HospitalManagementSystem/blob/main/NLP_Project_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [3]:
import os
import string
import spacy
import pickle  # To save/load preprocessed dataset
from datasets import load_dataset
from spacy.lang.en.stop_words import STOP_WORDS

In [4]:
CACHE_FILE = "/content/processed_dataset.pkl"

In [5]:
nlp = spacy.load("en_core_web_sm")

In [6]:
def extract_content_words(text):
    """
    Extracts content words (nouns, verbs, adjectives, adverbs) while:
    - Removing stopwords
    - Removing punctuation
    """
    doc = nlp(text)
    content_words = [
        token.lemma_.lower() for token in doc
        if token.pos_ in ["NOUN", "VERB", "ADJ", "ADV"]
        and token.text.lower() not in STOP_WORDS
        and token.text not in string.punctuation
    ]
    return " ".join(content_words)

In [7]:
def process_sample(entry):
    """
    Extracts content words separately for 'context' and 'lyrics'.
    """
    full_text = entry["lyrics"]

    if "### context:" in full_text and "### lyrics:" in full_text:
        parts = full_text.split("### lyrics:")
        context_text = parts[0].replace("### context:", "").strip()
        lyrics_text = parts[1].strip()
    else:
        context_text, lyrics_text = "", full_text

    return {
        "context_content_words": extract_content_words(context_text),
        "lyrics_content_words": extract_content_words(lyrics_text)
    }

In [8]:
# Load dataset only if not processed before
if os.path.exists(CACHE_FILE):
    print("🔹 Loading preprocessed dataset from cache...")
    with open(CACHE_FILE, "rb") as f:
        dataset = pickle.load(f)
else:
    print(" Processing dataset (This runs only once)...")
    dataset_name = "kibru/rap-lyrics-v3"
    dataset = load_dataset(dataset_name)
    dataset = dataset.map(process_sample)

    # Save processed dataset to cache
    with open(CACHE_FILE, "wb") as f:
        pickle.dump(dataset, f)

 Processing dataset (This runs only once)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/386 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.77M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7319 [00:00<?, ? examples/s]

In [9]:
def get_processed_sample(index):
    """Retrieve a processed sample by index."""
    return dataset["train"][index]

In [10]:
def show_processed_example():
    """Prompts user for an index and prints the processed sample in a structured format."""
    index = int(input("Enter sample index: "))
    sample = get_processed_sample(index)

    print("\n Processed Sample (Index: {})".format(index))
    print("-" * 50)
    for key, value in sample.items():
        print(f"-> {key.capitalize()}:\n{value}\n")
    print("-" * 50)

show_processed_example()

Enter sample index: 0

 Processed Sample (Index: 0)
--------------------------------------------------
-> Text:
I shoot the lights out
Hide 'til it's bright out
Oh, just another lonely night
Are you willing to sacrifice your life?

-> Num_tokens:
29

-> Completion:
A person is going through a dark and difficult time in their life, seeking comfort and solace in the darkness, and questioning if they are willing to let go of their current struggles to find a brighter future.

-> Lyrics:
### context: A person is going through a dark and difficult time in their life, seeking comfort and solace in the darkness, and quest
 ### lyrics: I shoot the lights out
Hide 'til it's bright out
Oh, just another lonely night
Are you willing to sacrifice your life?

-> Context_content_words:
person go dark difficult time life seek comfort solace darkness quest

-> Lyrics_content_words:
shoot light hide bright lonely night willing sacrifice life

--------------------------------------------------


In [11]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [12]:
from collections import Counter
import itertools
import random
import torch
from torch.utils.data import Dataset, DataLoader

In [13]:
# Build vocab from context and lyrics
def build_vocab(dataset, min_freq=2):
    counter = Counter()
    for sample in dataset["train"]:
        tokens = sample["context_content_words"].split() + sample["lyrics_content_words"].split()
        counter.update(tokens)

    # Special tokens
    vocab = {
        "[PAD]": 0,
        "[SOS]": 1,
        "[EOS]": 2,
        "[MASK]": 3,
        "[UNK]": 4
    }

    index = len(vocab)
    for word, freq in counter.items():
        if freq >= min_freq and word not in vocab:
            vocab[word] = index
            index += 1

    return vocab

# Build and save vocab
vocab = build_vocab(dataset)
id2word = {i: w for w, i in vocab.items()}
word2id = vocab

In [14]:
def corrupt_input(text, method="masking", mask_prob=0.15):
    tokens = text.split()

    if method == "masking":
        corrupted = [
            "[MASK]" if random.random() < mask_prob else tok
            for tok in tokens
        ]
    elif method == "shuffling":
        corrupted = tokens[:]
        random.shuffle(corrupted)
    elif method == "deletion":
        corrupted = [tok for tok in tokens if random.random() > mask_prob]
    elif method == "insertion":
        insert_words = random.choices(list(word2id.keys()), k=int(mask_prob * len(tokens)))
        corrupted = tokens + insert_words
        random.shuffle(corrupted)
    else:
        corrupted = tokens

    return corrupted

In [15]:
class RapLyricsDataset(Dataset):
    def __init__(self, data, vocab, max_len=100, corrupt_method="masking"):
        self.data = data["train"]
        self.vocab = vocab
        self.max_len = max_len
        self.corrupt_method = corrupt_method

    def encode(self, tokens, add_special_tokens=True):
        ids = [self.vocab.get(tok, self.vocab["[UNK]"]) for tok in tokens]
        if add_special_tokens:
            ids = [self.vocab["[SOS]"]] + ids + [self.vocab["[EOS]"]]
        return ids[:self.max_len] + [self.vocab["[PAD]"]] * (self.max_len - len(ids))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        clean = sample["lyrics_content_words"]
        noisy = " ".join(corrupt_input(sample["context_content_words"], self.corrupt_method))

        input_ids = torch.tensor(self.encode(noisy.split()))
        target_ids = torch.tensor(self.encode(clean.split()))

        return {"input_ids": input_ids, "target_ids": target_ids}

# Create dataset using existing dataset and vocab
train_dataset = RapLyricsDataset(
    data=dataset,
    vocab=vocab,
    max_len=100,
    corrupt_method="masking"  # or "shuffling", "noise"
)

In [16]:
import torch.nn as nn
import torch.nn.functional as F

In [17]:
class TransformerLyricsGenerator(nn.Module):
    def __init__(self, vocab_size, embed_dim=256, num_heads=8, num_layers=6, ff_dim=512, dropout=0.1, max_len=100):
        super().__init__()
        self.embed_dim = embed_dim
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Embedding(max_len, embed_dim)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, nhead=num_heads, dim_feedforward=ff_dim, dropout=dropout, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.output_layer = nn.Linear(embed_dim, vocab_size)

    def forward(self, input_ids):
        batch_size, seq_len = input_ids.shape
        positions = torch.arange(0, seq_len, device=input_ids.device).unsqueeze(0).expand(batch_size, seq_len)

        x = self.token_embedding(input_ids) + self.pos_embedding(positions)
        x = self.transformer_encoder(x)
        logits = self.output_layer(x)
        return logits

In [18]:
import torch.optim as optim
import time

In [21]:
# Hyperparameters
EPOCHS = 50
BATCH_SIZE = 16
LEARNING_RATE = 3e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate model
vocab_size = len(vocab)  # `vocab` is defined in previous blocks
model = TransformerLyricsGenerator(vocab_size).to(DEVICE)

PAD_TOKEN_ID = vocab["[PAD]"]

# Optimizer and Loss
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN_ID)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Training Loop with time logging
for epoch in range(EPOCHS):
    print(f"\n Epoch {epoch+1}/{EPOCHS}")
    start_time = time.time()

    model.train()
    total_loss = 0

    for batch in train_loader:
        corrupted_input = batch["input_ids"].to(DEVICE)
        target_output = batch["target_ids"].to(DEVICE)

        optimizer.zero_grad()
        output_logits = model(corrupted_input)

        # Reshape to calculate loss
        loss = criterion(output_logits.view(-1, vocab_size), target_output.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    epoch_time = time.time() - start_time
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} completed | Time: {epoch_time:.2f} sec | Avg Loss: {avg_loss:.4f}")


 Epoch 1/50
Epoch 1 completed | Time: 12.95 sec | Avg Loss: 6.6170

 Epoch 2/50
Epoch 2 completed | Time: 12.90 sec | Avg Loss: 6.4256

 Epoch 3/50
Epoch 3 completed | Time: 12.93 sec | Avg Loss: 6.3834

 Epoch 4/50
Epoch 4 completed | Time: 12.93 sec | Avg Loss: 6.3391

 Epoch 5/50
Epoch 5 completed | Time: 13.30 sec | Avg Loss: 6.2857

 Epoch 6/50
Epoch 6 completed | Time: 13.05 sec | Avg Loss: 6.2126

 Epoch 7/50
Epoch 7 completed | Time: 13.11 sec | Avg Loss: 6.1105

 Epoch 8/50
Epoch 8 completed | Time: 13.32 sec | Avg Loss: 5.9762

 Epoch 9/50
Epoch 9 completed | Time: 13.58 sec | Avg Loss: 5.8271

 Epoch 10/50
Epoch 10 completed | Time: 13.64 sec | Avg Loss: 5.6519

 Epoch 11/50
Epoch 11 completed | Time: 13.55 sec | Avg Loss: 5.4625

 Epoch 12/50
Epoch 12 completed | Time: 13.41 sec | Avg Loss: 5.2640

 Epoch 13/50
Epoch 13 completed | Time: 13.29 sec | Avg Loss: 5.0536

 Epoch 14/50
Epoch 14 completed | Time: 13.32 sec | Avg Loss: 4.8595

 Epoch 15/50
Epoch 15 completed | Tim

In [24]:
def generate_lyrics(model, context, vocab, max_len=50, temperature=1.0):
    model.eval()
    tokens = corrupt_input(context, method="masking")
    input_ids = [vocab.get(tok, vocab["[UNK]"]) for tok in tokens]
    input_ids = [vocab["[SOS]"]] + input_ids
    input_tensor = torch.tensor(input_ids).unsqueeze(0).to(DEVICE)

    generated = input_ids

    with torch.no_grad():
        for _ in range(max_len):
            output = model(torch.tensor([generated]).to(DEVICE))
            next_token_logits = output[0, -1, :] / temperature
            probs = torch.nn.functional.softmax(next_token_logits, dim=-1)

            next_token_id = torch.multinomial(probs, num_samples=1).item()

            if next_token_id == vocab["[EOS]"]:
                break

            generated.append(next_token_id)

    return " ".join([id2word.get(tok, "[UNK]") for tok in generated[1:]])


In [25]:
context = "midnight thoughts under city lights"
print("Input context:", context)

generated = generate_lyrics(model, context, vocab)
print("\nGenerated Lyrics:\n", generated)

Input context: midnight thoughts under city lights

Generated Lyrics:
 midnight [UNK] [MASK] city [UNK] pull hate girl girl lose plan middle lot


In [26]:
torch.save(model.state_dict(), "rap_lyrics_model.pt")

In [27]:
import torch
import json

def save_model(model, vocab, path_prefix="rap_lyrics_model"):
    # Save model weights
    torch.save(model.state_dict(), f"{path_prefix}.pt")

    # Save vocab
    with open(f"{path_prefix}_vocab.json", "w") as f:
        json.dump(vocab, f)

    print(f"✅ Model and vocab saved as {path_prefix}.pt and {path_prefix}_vocab.json")
